In [15]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

# Coutry Code
ccodes = pd.read_csv("countries.csv", sep=" \t")
app.layout = html.Div([
  html.Label("Country Code"),
  dcc.Dropdown(id="country-dropdown", 
               options=ccodes["A2"], 
               value="DE",
               #options=[{"label": row["Country"], "value": row["A2"]} for _, row in ccodes.iterrows()]
               ),
  html.Label("Zip Code"),
  dcc.Input(id="zip-input", type="number", placeholder="Enter a Zip Code", debounce=0.5),

  dcc.Graph(id="weather-graph")
])

@memory.cache
def fetch_temperatures(country_code, zip_code, start, end):
  lat, lon = geopos_from_zipcode(country_code, zip_code)
  if lat is None or lon is None:
    return pd.DataFrame(columns=["temp"])
  
  location = Point(lat, lon, 500)
  data = Hourly(location, start, end)
  data = data.fetch()
  return data

@app.callback(
  Output("weather-graph", "figure"),
  Input("country-dropdown", "value"),
  Input("zip-input", "value")
)
def update_graph(country_code, zip_code):
  if country_code is None or zip_code is None:
    return {}
  
  # Set time period
  start = datetime(2021, 1, 1)
  end = datetime(2022, 12, 31)
  
  data = fetch_temperatures(country_code, zip_code, start, end)
  return px.line(data, y='temp')


In [ ]:
app.run_server()

In [16]:
import co2intensity

co2intensity.load_all()

Date


,Date,Biomass [MWh] Calculated resolutions,Hydropower [MWh] Calculated resolutions,Wind offshore [MWh] Calculated resolutions,Wind onshore [MWh] Calculated resolutions,Photovoltaics [MWh] Calculated resolutions,Other renewable [MWh] Calculated resolutions,Nuclear [MWh] Calculated resolutions,Lignite [MWh] Calculated resolutions,Hard coal [MWh] Calculated resolutions,...,Other renewable [g CO2eq/kWh],Nuclear [g CO2eq/kWh],Lignite [g CO2eq/kWh],Hard coal [g CO2eq/kWh],Fossil gas [g CO2eq/kWh],Hydro pumped storage [g CO2eq/kWh],Other conventional [g CO2eq/kWh],CO2 sum,kWh sum,Intensity
0,2018-01-01 00:00:00,4704.50,1783.75,2914.25,28503.50,0.0,168.50,5853.75,6723.75,1711.00,...,8425.0,30029.7375,7748449.5,1971756.4,1589568.75,92429.2,894425.0,1.481578e+07,56358.00,262.886889
1,2018-01-01 01:00:00,4673.25,1676.25,2997.75,29638.00,0.0,168.00,4754.00,6810.00,1609.25,...,8400.0,24388.0200,7847844.0,1854499.7,1589568.75,186790.2,882175.0,1.487484e+07,56621.50,262.706500
2,2018-01-01 02:00:00,4666.75,1672.00,3093.25,30173.75,0.0,167.25,4591.00,6765.75,1683.25,...,8362.5,23551.8300,7796850.3,1939777.3,1584283.95,119548.7,850500.0,1.480910e+07,56828.50,260.592821
3,2018-01-01 03:00:00,4665.25,1658.25,3134.50,31021.50,0.0,166.75,4888.75,6728.00,1679.50,...,8337.5,25079.2875,7753347.2,1935455.8,1595018.70,14860.0,844550.0,1.467327e+07,57613.50,254.684555
4,2018-01-01 04:00:00,4682.25,1654.00,3209.50,31015.00,0.0,166.00,4696.50,6709.75,1677.50,...,8300.0,24093.0450,7732315.9,1933151.0,1592376.30,12779.6,865025.0,1.467294e+07,57499.75,255.182732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43843,2023-01-01 19:00:00,4678.75,1216.00,6006.25,19164.25,0.0,131.00,2458.75,4513.25,2377.50,...,6550.0,12613.3875,5201069.3,2739831.0,1083879.45,846425.6,864850.0,1.313982e+07,46270.00,283.981356
43844,2023-01-01 20:00:00,4624.00,1197.25,5592.75,19560.25,0.0,129.00,2462.25,4456.00,2196.50,...,6450.0,12631.3425,5135094.4,2531246.6,1104358.05,516459.3,867300.0,1.253368e+07,44866.50,279.354897
43845,2023-01-01 21:00:00,4573.25,1176.75,6097.00,19429.50,0.0,129.00,2461.75,4518.50,1913.25,...,6450.0,12628.7775,5207119.4,2204829.3,1081071.90,446617.3,861525.0,1.216258e+07,44669.00,272.282363
43846,2023-01-01 22:00:00,4512.25,1211.50,5142.00,18481.00,0.0,129.25,2461.50,4530.00,1897.25,...,6462.5,12627.4950,5220372.0,2186390.9,1098742.95,390520.8,860125.0,1.206718e+07,42570.75,283.461848


In [2]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import pandas as pd
import heatpump as hf
import heatdemand as hd

import datasource
from datetime import datetime, date
import numpy as np
start = datetime(2021, 1, 1)
end = datetime(2022, 12, 31)

df = datasource.fetch_all(country_code='DE',zip_code=81829,start=start,end=end)

# compute COP and electrical Power
df = hf.compute_cop(df,"Carnot")
df = hd.heat_demand(df, b_type="single_family_house", b_age="1984-94",t_design=-15, A=120)
df = hf.compute_P_electrical(df)



Date


In [5]:
C = 328000e3     # J/K
#U = 0.1     # W/(m2*K)
#A_liv = 120     # m2
#UA = U*A_liv
UA = 582
timestep = 3600 # h

df["InternalEnergy"] = 15*C # 15°C to J
for i in range(len(df)-1):
    Q_dot = (df["temp"].iloc[i] - df["InternalEnergy"].iloc[i]/C)*UA # W
    #Q_dot += 100 # 100W heating for each habitant
    Q_dot += df["Q_dot_H"].iloc[i] # heat pump
    df["InternalEnergy"].iloc[i+1] = df["InternalEnergy"].iloc[i] + Q_dot*timestep

df["InternalTemperature"] = df["InternalEnergy"]/C # J to °C
px.line(df, y=["InternalTemperature", "temp"])

/home/missinguser/CSE/fa2023/venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

